---
<h1><center> 6. Capstone Modelling 3.0 </center></h1>


--- 

Through all three modelling notebooks I was unable to find a model, set of features or hyperparameters that would significantly improve my log loss. I am aware my modelling notebooks are not neat or commented. I didn't have time to format them as I would've liked. Before I upload my capstone to my github I will need to clean and annotate my modelling & evaluation process. I believe a simple logistic regression is able to accomlish my original objective of defining the quality of a shot, however none of my models were able to generate a low log loss score as it heavily penalized situations where a shot has a high probability of being made and misses or conversely has a low probability of being made and goes in. Players can get lucky or unlucky regardless of whether a shot is probable to be a make. 

In [120]:
# packages used
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn.metrics import log_loss

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import brier_score_loss

from xgboost import XGBClassifier
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import shap

In [92]:
data = pd.read_csv('capstone_data_V3.csv')
pd.set_option('display.max_columns', 500)
print(data.shape)
data.head()

(125466, 36)


,ACTION_TYPE,PLAYER_NAME,PLAYER_ID,TEAM_NAME,PRIMARY_POSITION,HEIGHT,WEIGHT,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,PTS_TYPE,SHOT_RESULT_made,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,CLOSE_DEF_DIST,CLOSE_DEF_NAME,CLOSE_DEF_ID,CLOSE_DEF_HEIGHT,CLOSE_DEF_WEIGHT,CLOSE_DEF_POSITION,feet,inches,INCHES,feet1,inches1,DEF_INCHES,HEIGHT_DIFFERENCE,WEIGHT_DIFFERENCE,adjy,adjx,ANG
0,Jump,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,5.2,0,0.8,2,0,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,14,72,131,3.8,Kelly Olynyk,203482,7-0,240,4,6,11,83,7,0,84,-1,0,131,72,28
1,Jump,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,17.6,0,0.8,2,1,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,20,-84,186,7.3,Kelly Olynyk,203482,7-0,240,4,6,11,83,7,0,84,-1,0,186,84,24
2,Jump,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,1.6,0,0.8,2,1,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,54,34,1.7,Kelly Olynyk,203482,7-0,240,4,6,11,83,7,0,84,-1,0,34,54,57
3,Jump,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,10.6,0,0.8,2,0,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,17,-146,104,7.3,Kelly Olynyk,203482,7-0,240,4,6,11,83,7,0,84,-1,0,104,146,54
4,Jump,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,14.2,1,2.0,2,0,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,19,-51,190,3.9,Kelly Olynyk,203482,7-0,240,4,6,11,83,7,0,84,-1,0,190,51,15


In [93]:
# make a 3 point data frame 
# 2 point dataframe


# features so far action type, primary position, shot clock, dribbles,
# touch time, shot distance, close def distance, height difference, weight difference, shot angle


# pts-type?

In [94]:
# getting the X values for model
model_cols = ['ACTION_TYPE', 'PRIMARY_POSITION', 'SHOT_CLOCK', 'DRIBBLES', 'TOUCH_TIME',\
              'SHOT_DISTANCE', 'CLOSE_DEF_DIST', 'HEIGHT_DIFFERENCE', 'WEIGHT_DIFFERENCE', 'ANG']

df = data[model_cols]
print(df.shape)
df.head()

(125466, 10)


,ACTION_TYPE,PRIMARY_POSITION,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,SHOT_DISTANCE,CLOSE_DEF_DIST,HEIGHT_DIFFERENCE,WEIGHT_DIFFERENCE,ANG
0,Jump,4,5.2,0,0.8,14,3.8,-1,0,28
1,Jump,4,17.6,0,0.8,20,7.3,-1,0,24
2,Jump,4,1.6,0,0.8,6,1.7,-1,0,57
3,Jump,4,10.6,0,0.8,17,7.3,-1,0,54
4,Jump,4,14.2,1,2.0,19,3.9,-1,0,15


In [95]:
# dummies for categorical

dummy = ['ACTION_TYPE', 'PRIMARY_POSITION']

df = pd.get_dummies(df, columns=dummy)
print(df.shape)

(125466, 19)


In [96]:
# getting X and Y values
X = df
Y = data['SHOT_RESULT_made']

print(X.shape)
print(Y.shape)

(125466, 19)
(125466,)


In [97]:
# splitting data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, stratify=shot_results, random_state=0)

In [99]:
rf = ensemble.RandomForestClassifier(n_estimators=300, random_state = 1)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [100]:
train_rf_probs = rf.predict_proba(X_train)[:, 1]
rf_probs = rf.predict_proba(X_test)[:, 1]

In [101]:
train_loss = log_loss(y_train, train_rf_probs)
test_loss = log_loss(y_test, rf_probs)
print(train_loss)
print(test_loss)

0.18194080477270147
0.6451350330776909


In [102]:
# scale
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# reduce dimenionality
my_pca = PCA(.95)
pca_X_train = my_pca.fit_transform(X_train)
pca_X_test = my_pca.transform(X_test)

print(my_pca.n_components_)
print(my_pca.explained_variance_)

14
[3.02220246 2.92989554 1.61971949 1.40880712 1.28066213 1.26127525
 1.17304228 1.1246335  1.03385626 0.97770749 0.81712975 0.69730289
 0.62420477 0.55093562]


In [104]:
param_grid = {'C': [0.00001, 0.0001, 0.001, 0.01, 1, 10, 100, 1000, 10000, 100000] }

grid_search = GridSearchCV(LogisticRegression(), param_grid)

fitted_search = grid_search.fit(pca_X_train, y_train)



In [105]:
fitted_search.best_estimator_

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [107]:
train_probs = fitted_search.best_estimator_.predict_proba(pca_X_train)[:, 1]
test_probs = fitted_search.best_estimator_.predict_proba(pca_X_test)[:, 1]
train_loss = log_loss(y_train, train_probs)
test_loss = log_loss(y_test, test_probs)
train_brier = brier_score_loss(y_train, train_probs)
test_brier = brier_score_loss(y_test, test_probs)
print(train_loss)
print(test_loss)
print(train_brier) 
print(test_brier)

0.641310403122137
0.6431759161150561
0.22591324159799056
0.2266179654484315


In [108]:
param_grid = {'C': [0.00001, 0.0001, 0.001, 0.01, 1, 10, 100, 1000, 10000, 100000] }

grid_search = GridSearchCV(LogisticRegression(), param_grid)

fitted_search = grid_search.fit(X_train, y_train)

In [109]:
fitted_search.best_estimator_

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [110]:
train_probs = fitted_search.best_estimator_.predict_proba(X_train)[:, 1]
test_probs = fitted_search.best_estimator_.predict_proba(X_test)[:, 1]
train_loss = log_loss(y_train, train_probs)
test_loss = log_loss(y_test, test_probs)
train_brier = brier_score_loss(y_train, train_probs)
test_brier = brier_score_loss(y_test, test_probs)
print(train_loss)
print(test_loss)
print(train_brier) 
print(test_brier)

0.6412102130813541
0.6430159040018452
0.22587077469783726
0.2265504325415545


In [112]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
data['SHOT_QUALITY'] = fitted_search.best_estimator_.predict_proba(X_scaled)[:, 1]

In [117]:
data.head(10)

,ACTION_TYPE,PLAYER_NAME,PLAYER_ID,TEAM_NAME,PRIMARY_POSITION,HEIGHT,WEIGHT,SHOT_CLOCK,DRIBBLES,TOUCH_TIME,PTS_TYPE,SHOT_RESULT_made,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,CLOSE_DEF_DIST,CLOSE_DEF_NAME,CLOSE_DEF_ID,CLOSE_DEF_HEIGHT,CLOSE_DEF_WEIGHT,CLOSE_DEF_POSITION,feet,inches,INCHES,feet1,inches1,DEF_INCHES,HEIGHT_DIFFERENCE,WEIGHT_DIFFERENCE,adjy,adjx,ANG,SHOT_QUALITY
0,Jump,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,5.2,0,0.8,2,0,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.,14,72,131,3.8,Kelly Olynyk,203482,7-0,240,4,6,11,83,7,0,84,-1,0,131,72,28,0.345000
1,Jump,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,17.6,0,0.8,2,1,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,20,-84,186,7.3,Kelly Olynyk,203482,7-0,240,4,6,11,83,7,0,84,-1,0,186,84,24,0.417646
2,Jump,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,1.6,0,0.8,2,1,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,6,54,34,1.7,Kelly Olynyk,203482,7-0,240,4,6,11,83,7,0,84,-1,0,34,54,57,0.334198
3,Jump,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,10.6,0,0.8,2,0,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,17,-146,104,7.3,Kelly Olynyk,203482,7-0,240,4,6,11,83,7,0,84,-1,0,104,146,54,0.418967
4,Jump,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,14.2,1,2.0,2,0,2PT Field Goal,Mid-Range,Center(C),16-24 ft.,19,-51,190,3.9,Kelly Olynyk,203482,7-0,240,4,6,11,83,7,0,84,-1,0,190,51,15,0.339060
5,Jump,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,10.5,1,1.5,2,0,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.,19,-172,83,4.2,Kelly Olynyk,203482,7-0,240,4,6,11,83,7,0,84,-1,0,83,172,64,0.348774
6,Jump,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,16.0,0,0.8,2,1,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,19,-147,129,5.5,Kelly Olynyk,203482,7-0,240,4,6,11,83,7,0,84,-1,0,129,147,48,0.385932
7,Jump,Kevin Garnett,708,Brooklyn Nets,4,6-11,240,14.5,1,1.4,2,1,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.,18,-125,132,4.1,Kelly Olynyk,203482,7-0,240,4,6,11,83,7,0,84,-1,0,132,125,43,0.357484
8,Jump,Kobe Bryant,977,Los Angeles Lakers,2,6-6,212,15.9,0,0.9,3,0,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.,25,147,206,8.5,Kelly Olynyk,203482,7-0,240,4,6,6,78,7,0,84,-6,-28,206,147,35,0.411580
9,Jump,Tim Duncan,1495,San Antonio Spurs,5,6-11,250,3.4,0,0.6,2,0,2PT Field Goal,In The Paint (Non-RA),Center(C),Less Than 8 ft.,4,-48,9,3.0,Kelly Olynyk,203482,7-0,240,4,6,11,83,7,0,84,-1,10,9,48,79,0.356875


In [118]:
# data.to_csv('capstone_data_V4.csv', index = False)

In [80]:
from sklearn.metrics import classification_report

y_pred = fitted_search.best_estimator_.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.63      0.77      0.69     20592
           1       0.62      0.46      0.53     17048

   micro avg       0.63      0.63      0.63     37640
   macro avg       0.63      0.61      0.61     37640
weighted avg       0.63      0.63      0.62     37640



In [86]:
XGB_model = XGBClassifier()
XGB_model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [87]:
train_probs = XGB_model.predict_proba(X_train)[:, 1]
test_probs = XGB_model.predict_proba(X_test)[:, 1]
train_loss = log_loss(y_train, train_probs)
test_loss = log_loss(y_test, test_probs)
print(train_loss)
print(test_loss)

0.634682174412744
0.6414814919062348


Well I've lost my mind.... 3 days of trying ti find a model that could minimize the log loss was unsuccessful. My rationale as to why I'm having terrible scores are players can still make bad shots, and players can still miss good shots. The purpose of my analysis was to define what is a good shot and conversly what is a bad shot. I've wasted so much time trying to find a better model and I'm extremely discouraged.